In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 1. Get Event Details

In [18]:
# loop from page 1 to page 34 (Date 24 Jan 2024), page url examlpes: 'https://visitseattle.org/events/page/1'

pages = range(1,35)
events = []
for page in pages:
    url = 'https://visitseattle.org/events/page/' + str(page)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    selector = 'div.search-result-preview > div > h3 > a'
    a_eles=soup.select(selector)
    events = events + [x['href'] for x in a_eles]

events

['https://visitseattle.org/events/aryana-leon/',
 'https://visitseattle.org/events/colorworks/',
 'https://visitseattle.org/events/dar-williams/',
 'https://visitseattle.org/events/deb-perelman/',
 'https://visitseattle.org/events/emanuel-brown/',
 'https://visitseattle.org/events/james-miles/',
 'https://visitseattle.org/events/madeline-pendleton/',
 'https://visitseattle.org/events/october-london/',
 'https://visitseattle.org/events/tacoma-stars-vs-empire-strykers/',
 'https://visitseattle.org/events/dungeons-and-drag-queens/',
 'https://visitseattle.org/events/global-rhythms-yungchen-lhamo/',
 'https://visitseattle.org/events/harlem-globetrotters-2024-world-tour/',
 'https://visitseattle.org/events/seattle-kraken-vs-st-louis-blues/',
 'https://visitseattle.org/events/shelf-nunny/',
 'https://visitseattle.org/events/super-diamond-the-neil-diamond-tribute-band/',
 'https://visitseattle.org/events/the-disorderlies-2/',
 'https://visitseattle.org/events/compania-nacional-de-danza/',
 'h

In [25]:
eventdata = []

for event in events:
    res = requests.get(event)

    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        
        name = soup.select_one('div.medium-6.columns.event-top > h1')
        date_time = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)')
        location = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)')
        event_type = soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)')
        region = soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)')

        eventdata.append({
            "Name": name.get_text(strip=True) if name else "Not found",
            "Date & Time": date_time.get_text(strip=True) if date_time else "Not found",
            "Location": location.get_text(strip=True) if location else "Not found",
            "Type": event_type.get_text(strip=True) if event_type else "Not found",
            "Region": region.get_text(strip=True) if region else "Not found"
        })

        # eventdata.append({
        #     "Name": name,
        #     "Date & Time": date_time,
        #     "Location": location,
        #     "Type": event_type,
        #     "Region": region
        # })


df = pd.DataFrame(eventdata)


df.to_csv("events.csv", index=False)

NameError: name 'events' is not defined

# 2. Get Weather Info

In [9]:
url_weather = "https://api.weather.gov/points/39.7456,-97.0892"
res = requests.get(url_weather)
res

<Response [200]>

In [10]:
res.text

'{\n    "@context": [\n        "https://geojson.org/geojson-ld/geojson-context.jsonld",\n        {\n            "@version": "1.1",\n            "wx": "https://api.weather.gov/ontology#",\n            "s": "https://schema.org/",\n            "geo": "http://www.opengis.net/ont/geosparql#",\n            "unit": "http://codes.wmo.int/common/unit/",\n            "@vocab": "https://api.weather.gov/ontology#",\n            "geometry": {\n                "@id": "s:GeoCoordinates",\n                "@type": "geo:wktLiteral"\n            },\n            "city": "s:addressLocality",\n            "state": "s:addressRegion",\n            "distance": {\n                "@id": "s:Distance",\n                "@type": "s:QuantitativeValue"\n            },\n            "bearing": {\n                "@type": "s:QuantitativeValue"\n            },\n            "value": {\n                "@id": "s:value"\n            },\n            "unitCode": {\n                "@id": "s:unitCode",\n                "@typ

In [12]:
point_dict = res.json() # json is a dictionary/method. text is a sttribute
point_dict

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/39.7456,-97.0892',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-97.0892, 39.7456]},
 'properties': {'@id': 'https://api.weather.gov/p

In [13]:
point_dict.keys()

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])

In [14]:
forcast_url = point_dict['properties']['forecast']
forcast_url

'https://api.weather.gov/gridpoints/TOP/32,81/forecast'

In [15]:
res = requests.get(forcast_url)
res.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-97.1089731, 39.7668263],
    [-97.1085269, 39.7447788],
    [-97.0798467, 39.7451195],
    [-97.08028680000001, 39.767167],
    [-97.1089731, 39.7668263]]]},
 'properties': {'updated': '2024-01-28T00:10:15+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-01-28T00:57:18+00:00',
  'updateTime': '2024-01-28T00:10:15+00:00',
  'validTimes': '2024-01-27T18:00:00+00:00/P7DT10H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 441.96},
  'periods': [{'number': 1,
    'name': 'Tonight',
    'startTime': '2024-01-27T18:00:00-06:00',
    'endTime': '2024-01-28T06:00:00-06:00',
    'isDa

### Get Location Lat & Lon

In [44]:
# location name to lat lon

base_url = "https://nominatim.openstreetmap.org/search.php"

# use dictionary to represent query params
# use requests.get(url, params=query_params) to attach the query param dict
query_params = {
    "q": "Wallingford, Seattle",
    "format": "jsonv2"
}

res = requests.get(base_url, params=query_params)
res.json()

[{'place_id': 312844859,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 150973716,
  'lat': '47.6594631',
  'lon': '-122.3343417',
  'category': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.3131791431958073,
  'addresstype': 'suburb',
  'name': 'Wallingford',
  'display_name': 'Wallingford, Seattle, King County, Washington, 98015, United States',
  'boundingbox': ['47.6394631', '47.6794631', '-122.3543417', '-122.3143417']}]

In [23]:
# # get lat lon from location name
# # Function to get latitude and longitude for a location name
# def get_lat_lon(location_name):

#     # Base URL for OpenStreetMap API
#     base_url = "https://nominatim.openstreetmap.org/search.php"

#     query_params = {
#         "q": location_name,
#         "format": "jsonv2"
#     }
#     response = requests.get(base_url, params=query_params)
#     if response.status_code == 200 and len(response.json()) > 0:
#         data = response.json()[0]
#         return data.get('lat'), data.get('lon')
#     else:
#         return None, None

# # Read CSV file
# csv_file = 'events.csv' 
# df = pd.read_csv(csv_file)

# # Adding new columns for latitude and longitude
# df['Latitude'] = None
# df['Longitude'] = None

# for index, row in df.iterrows():
#     lat, lon = get_lat_lon(row['Location'])
#     if lat and lon:
#         df.at[index, 'Latitude'] = lat
#         df.at[index, 'Longitude'] = lon
        
# df.to_csv('events_latlon.csv', index=False)


In [45]:
import requests
import pandas as pd

def get_lat_lon(location_name, city='Seattle', state='WA', country='USA'):
    base_url = "https://nominatim.openstreetmap.org/search.php"
    query_params = {
        "q": f"{location_name}, {city}, {state}, {country}",  # More specific query
        "format": "jsonv2"
    }
    response = requests.get(base_url, params=query_params)
    if response.status_code == 200 and len(response.json()) > 0:
        data = response.json()[0]
        lat, lon = float(data.get('lat')), float(data.get('lon'))
        # Validate coordinates (example range for Seattle)
        if 47.4 <= lat <= 47.8 and -122.5 <= lon <= -122.2:
            return lat, lon
        else:
            return None, None  # Coordinates are outside the expected range
    else:
        return None, None  # API request failed or returned no data

# Read CSV file
csv_file = 'events.csv' 
df = pd.read_csv(csv_file)

# Adding new columns for latitude and longitude
df['Latitude'] = None
df['Longitude'] = None

for index, row in df.iterrows():
    lat, lon = get_lat_lon(row['Location'])
    if lat is not None and lon is not None:
        df.at[index, 'Latitude'] = lat
        df.at[index, 'Longitude'] = lon
    else:
        # Handle cases where coordinates are not found or invalid
        print(f"Coordinates not found or invalid for location: {row['Location']}")

# Save the updated DataFrame to a new CSV file
df.to_csv('events_latlon_2.csv', index=False)


Coordinates not found or invalid for location: The Neptune Theatre
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinates not found or invalid for location: Meany Center for the Performing Arts
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinates not found or invalid for location: Fountainhead Gallery
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinates not found or invalid for location: The Moore Theatre
Coordinates not found or invalid for location: Bainbridge Island Museum of Art
Coordinates not found or invalid for location: The Neptune Theatre
Coordinates not found or invalid for location: Meany Center for the Performing Arts
Coordinates not found or invalid for location: Funhouse
Coordinates not found or invalid for location: accesso ShoWare Center
Coordinate

### Use Lat & Lon Fetch Weather Info

In [46]:
import pandas as pd
from datetime import datetime
import requests

# Function to get weather information for latitude, longitude, and daytime of the event

from datetime import datetime

def get_weather_forecast(lat, lon, event_date):
    try:
        url_weather = f"https://api.weather.gov/points/{lat},{lon}"
        response = requests.get(url_weather)
        if response.status_code == 200:
            # Extract the forecast URL from the response
            forecast_url = response.json()['properties']['forecast']
            forecast_response = requests.get(forecast_url)
            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()['properties']['periods']
                for period in forecast_data:
                    forecast_date = datetime.strptime(period['startTime'], '%Y-%m-%dT%H:%M:%S%z').date()
                    if forecast_date == event_date and 'daytime' in period['name'].lower():
                        weather = period['shortForecast']
                        temperature = period['temperature']
                        wind_speed = period['windSpeed']
                        wind_direction = period['windDirection']
                        return weather, temperature, wind_speed, wind_direction
        return None, None, None, None
    except Exception as e:
        print(f"Error fetching weather data for lat: {lat}, lon: {lon}, date: {event_date}. Error: {e}")
        return None, None, None, None

# Function to get the latest weather forecast
def get_latest_weather_forecast(lat, lon):
    try:
        url_weather = f"https://api.weather.gov/points/{lat},{lon}"
        response = requests.get(url_weather)
        if response.status_code == 200:
            forecast_url = response.json()['properties']['forecast']
            forecast_response = requests.get(forecast_url)
            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()['properties']['periods'][0] # Get the latest forecast
                return forecast_data['shortForecast'], forecast_data['temperature'], forecast_data['windSpeed'], forecast_data['windDirection']
        return 'Not available', 'Not available', 'Not available', 'Not available'
    except Exception as e:
        print(f"Error fetching latest weather data for lat: {lat}, lon: {lon}. Error: {e}")
        return 'Not available', 'Not available', 'Not available', 'Not available'

 # Use a central Seattle location for the default weather forecast for the events with no lat & lon or return no weather info
    
def get_seattle_weather_forecast():
   
    seattle_lat = '47.6062'
    seattle_lon = '-122.3321'
    return get_latest_weather_forecast(seattle_lat, seattle_lon)


csv_file = 'events_latlon_2.csv'
df = pd.read_csv(csv_file)

# Adding new columns for weather details
df['weather'] = None
df['temperature'] = None
df['wind_speed'] = None
df['wind_direction'] = None

for index, row in df.iterrows():
    lat = row.get('Latitude')
    lon = row.get('Longitude')
    date_str = row['Date & Time'].split(' ')[0]

    if pd.notna(lat) and pd.notna(lon):
        try:
            if date_str.lower() == 'now' or date_str.lower() == 'ongoing':
                weather_info = get_latest_weather_forecast(lat, lon)
            else:
                event_date = datetime.strptime(date_str, '%m/%d/%Y').date()
                weather_info = get_weather_forecast(lat, lon, event_date)

            # Check if weather info is not returned
            if not all(weather_info):
                weather_info = get_seattle_weather_forecast()  # Default to Seattle weather
        except Exception:
            weather_info = get_seattle_weather_forecast()  # Default to Seattle weather
    else:
        weather_info = get_seattle_weather_forecast()  # Default to Seattle weather

    # Update the DataFrame with the weather information
    df.at[index, 'weather'], df.at[index, 'temperature'], df.at[index, 'wind_speed'], df.at[index, 'wind_direction'] = weather_info

# Export the updated dataframe to a CSV file
df.to_csv('Seattle_Events_Detail_Forcast.csv', index=False)



In [47]:

# Drop the latitude and longitude columns
df_without_lat_lon = df.drop(['Latitude', 'Longitude'], axis=1)

# Save the updated DataFrame without latitude and longitude to a new CSV file
output_csv_file = 'events.csv'
df_without_lat_lon.to_csv(output_csv_file, index=False)